In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
import pdb
from sklearn.metrics import roc_auc_score
np.random.seed(2020)
torch.manual_seed(2020)
import pdb

from dataset import load_data

from matrix_factorization import NCF, NCF_CVIB, NCF_IPS, NCF_SNIPS, NCF_DR

from utils import gini_index, ndcg_func, get_user_wise_ctr, rating_mat_to_sample, binarize, shuffle, minU
mse_func = lambda x,y: np.mean((x-y)**2)
acc_func = lambda x,y: np.sum(x == y) / len(x)

dataset_name = "coat"
# dataset_name = "yahoo"

if dataset_name == "coat":
    train_mat, test_mat = load_data("coat")        
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]

elif dataset_name == "yahoo":
    x_train, y_train, x_test, y_test = load_data("yahoo")
    x_train, y_train = shuffle(x_train, y_train)
    num_user = x_train[:,0].max() + 1
    num_item = x_train[:,1].max() + 1

print("# user: {}, # item: {}".format(num_user, num_item))
# binarize
y_train = binarize(y_train)
y_test = binarize(y_test)

===>Load from coat data set<===
[train] rating ratio: 0.080000
[test]  rating ratio: 0.053333
# user: 290, # item: 300


In [2]:
"NCF CVIB"
ncf_cvib = NCF_CVIB(num_user, num_item)
ncf_cvib.fit(x_train, y_train, lr=0.05, 
    alpha=10.0, gamma=1e-5, lamb=1e-3, tol=1e-6, 
    batch_size = 512, verbose=1)

test_pred = ncf_cvib.predict(x_test)
mse_ncf = mse_func(y_test, test_pred)
auc_ncf = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(ncf_cvib, x_test, y_test)

print("***"*5 + "[NCF-CVIB]" + "***"*5)
print("[NCF-CVIB] test mse:",mse_ncf)
print("[NCF-CVIB] test auc:", auc_ncf)
print("ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
    np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-CVIB]" + "***"*5)

[NCF-CVIB] epoch:0, xent:13.267024040222168
[NCF-CVIB] epoch:10, xent:27.240811705589294
[NCF-CVIB] epoch:17, xent:27.22674059867859
***************[NCF-CVIB]***************
[NCF-CVIB] test mse: 0.5825214462830888
[NCF-CVIB] test auc: 0.7700328227232691
ndcg@5:0.641975, ndcg@10:0.712083
Num User: 290
Gini index: 0.3297949064683352
Global utility: 0.5275862068965518
***************[NCF-CVIB]***************


In [3]:
"NCF naive"
ncf = NCF(num_user, num_item)
ncf.fit(x_train, y_train, lr=0.05, lamb=1e-3, tol=1e-6, 
    batch_size = 512, verbose=1)
test_pred = ncf.predict(x_test)
mse_ncf = mse_func(y_test, test_pred)
auc_ncf = roc_auc_score(y_test, test_pred)
ndcg_res = ndcg_func(ncf, x_test, y_test)

print("***"*5 + "[NCF]" + "***"*5)
print("[NCF] test mse:", mse_func(y_test, test_pred))
print("[NCF] test auc:", auc_ncf)
print("ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
    np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF]" + "***"*5)

[NCF] epoch:0, xent:8.990946412086487
[NCF] epoch:10, xent:6.643972098827362
[NCF] epoch:20, xent:6.232902467250824
[NCF] epoch:30, xent:6.119126111268997
[NCF] epoch:31, xent:6.125039279460907
***************[NCF]***************
[NCF] test mse: 0.20648528803268903
[NCF] test auc: 0.7608803325809124
ndcg@5:0.625427, ndcg@10:0.698200
Num User: 290
Gini index: 0.3485643817767238
Global utility: 0.5110344827586206
***************[NCF]***************


In [4]:
"NCF IPS"
ncf_ips = NCF_IPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

ncf_ips.fit(x_train, y_train, 
    y_ips=y_ips,
    lr=0.01,
    batch_size=512,
    lamb=1e-3,tol=1e-6, verbose=1)

ndcg_res = ndcg_func(ncf_ips, x_test, y_test)
test_pred = ncf_ips.predict(x_test)
mse_ncfips = mse_func(y_test, test_pred)
auc_ncfips = roc_auc_score(y_test, test_pred)
print("***"*5 + "[NCF-IPS]" + "***"*5)
print("[NCF-IPS] test mse:", mse_ncfips)
print("[NCF-IPS] test auc:", auc_ncfips)
print("ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
    np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-IPS]" + "***"*5)

[NCF-IPS] epoch:0, xent:114.86887741088867
[NCF-IPS] epoch:10, xent:80.96701860427856
[NCF-IPS] epoch:20, xent:73.84566402435303
[NCF-IPS] epoch:30, xent:69.79464864730835
[NCF-IPS] epoch:40, xent:65.42658805847168
[NCF-IPS] epoch:50, xent:63.507972240448
[NCF-IPS] epoch:51, xent:63.69289684295654
***************[NCF-IPS]***************
[NCF-IPS] test mse: 0.21128340528956277
[NCF-IPS] test auc: 0.75915065355459
ndcg@5:0.625268, ndcg@10:0.701324
Num User: 290
Gini index: 0.34621152510992903
Global utility: 0.5137931034482759
***************[NCF-IPS]***************


In [5]:
"NCF SNIPS"
ncf_snips = NCF_SNIPS(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

ncf_snips.fit(x_train, y_train, 
    y_ips=y_ips,
    lr=0.01,
    batch_size=512,
    lamb=1e-3,tol=1e-6, verbose=1)
ndcg_res = ndcg_func(ncf_snips, x_test, y_test)
test_pred = ncf_snips.predict(x_test)
mse_ncfsnips = mse_func(y_test, test_pred)
auc_ncfsnips = roc_auc_score(y_test, test_pred)
print("***"*5 + "[NCF-SNIPS]" + "***"*5)
print("[NCF-SNIPS] test mse:", mse_ncfsnips)
print("[NCF-SNIPS] test auc:", auc_ncfsnips)
print("ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
    np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-SNIPS]" + "***"*5)


[NCF-SNIPS] epoch:0, xent:8.995021224021912
[NCF-SNIPS] epoch:10, xent:7.806631803512573
[NCF-SNIPS] epoch:20, xent:6.2327945828437805
[NCF-SNIPS] epoch:30, xent:6.2342768013477325
[NCF-SNIPS] epoch:33, xent:6.199462562799454
***************[NCF-SNIPS]***************
[NCF-SNIPS] test mse: 0.19096356313095192
[NCF-SNIPS] test auc: 0.77325216775354
ndcg@5:0.638492, ndcg@10:0.712821
Num User: 290
Gini index: 0.3311930762608193
Global utility: 0.5248275862068966
***************[NCF-SNIPS]***************


In [6]:
"NCF DR"
ncf_dr = NCF_DR(num_user, num_item)

ips_idxs = np.arange(len(y_test))
np.random.shuffle(ips_idxs)
y_ips = y_test[ips_idxs[:int(0.05 * len(ips_idxs))]]

ncf_dr.fit(x_train, y_train, y_ips=y_ips, batch_size=512, lr=0.01, lamb=1e-3,verbose=0)
ndcg_res = ndcg_func(ncf_dr, x_test, y_test)

test_pred = ncf_dr.predict(x_test)
mse_mfdr = mse_func(y_test, test_pred)
auc_mfdr = roc_auc_score(y_test, test_pred)
print("***"*5 + "[NCF-DR]" + "***"*5)
print("[NCF-DR] test mse:", mse_mfdr)
print("[NCF-DR] test auc:", auc_mfdr)
print("ndcg@5:{:.6f}, ndcg@10:{:.6f}".format(
    np.mean(ndcg_res["ndcg_5"]), np.mean(ndcg_res["ndcg_10"])))
user_wise_ctr = get_user_wise_ctr(x_test,y_test,test_pred)
gi,gu = gini_index(user_wise_ctr)
print("***"*5 + "[NCF-DR]" + "***"*5)

ValueError: Using a target size (torch.Size([512])) that is different to the input size (torch.Size([512, 1])) is deprecated. Please ensure they have the same size.